<a href="https://colab.research.google.com/github/anuraagkansara/SecureNLP/blob/master/19jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import spacy

In [2]:
!git clone https://github.com/anuraagkansara/SecureNLP.git

fatal: destination path 'SecureNLP' already exists and is not an empty directory.


In [15]:
!pip install glove_python

     |████████████████████████████████| 266kB 2.6MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700348 sha256=79779a99a28c18e9537b5cf536a1a147385a072ae01e888bf64f3dca7d5f5ca5
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [3]:
!python -m spacy download en_core_web_md


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
import numpy as np
import os
import spacy
import nltk
import sklearn
# import pycrfsuite
# from mosestokenizer import MosesDetokenizer
from glove import Corpus, Glove
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from bs4 import BeautifulSoup
from imblearn.over_sampling import SMOTE


nlp = spacy.load('en_core_web_md')

In [0]:

def read_sentences(path, sentence_separator):
    filenames = os.listdir(path)
    result = []
    for filename in filenames:
        with open(path + filename) as file:
            lines = file.read()
        lines = lines.split(sentence_separator)[:-1]
        for line in lines:
            l = []
            for x in line.splitlines():
                w = x.split(' ')
                if w[0] == "''":
                    w[0] = '"'
                w[0] = w[0].lower()
                if len(w) == 2:
                    l.append(np.array(w))
            l = np.array(l)
            result.append(l)
    return result

In [0]:
train = read_sentences("SecureNLP/train/tokenized/", " \n")

In [0]:
test = []
for i in [1, 2, 3]:
    test.extend(read_sentences(f"SecureNLP/test_{i}/tokenized/", "\n\n"))

In [0]:
dev = read_sentences("SecureNLP/dev/tokenized/", " \n")

In [0]:
PATH = "SecureNLP/train/additional_plaintext/"
filenames = os.listdir(PATH)
additional_data = []
for filename in filenames:
    # print(filename)
    with open(PATH + filename) as file:
        lines = file.read()
    soup = BeautifulSoup(lines, 'html.parser')
    soup.get_text()
    for section in soup.find_all('section'):
        for section in section.contents:
            if not str(section).startswith("<") and str(section) != "\n":
                doc = nlp(str(section))
                for sent in doc.sents:
                    tmp = []
                    for token in sent:
                        if token.text and token.text not in ("\n", " "):
                            tmp.append(token.text.lower())
                    additional_data.append(tmp)

In [0]:
train_x = [list(x[:, 0]) for x in train]
train_y = [list(x[:, 1]) for x in train]

test_x = [list(x[:, 0]) for x in test]
test_y = [list(x[:, 1]) for x in test]

dev_x = [list(x[:, 0]) for x in dev]
dev_y = [list(x[:, 1]) for x in dev]

embed = [list(x[:, 0]) for x in train]
embed.extend(additional_data)

In [18]:
corpus = Corpus() 
corpus.fit(embed, window=10)

glove = Glove(no_components=100, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [0]:
def sentence_embodding(sentence):
    sums = np.zeros(100)
    for word in sentence:
        try:
            emb = glove.word_vectors[glove.dictionary[word]]
        except KeyError:
            emb = np.zeros(100)
        sums += emb
    return sums/len(sentence)
def get_labels(tags):
    if sum([tag == 'O' for tag in tags]) == len(tags):
        return 0
    return 1

In [0]:
X_train = np.array([sentence_embodding(s) for s in train_x])
y_train = np.array([get_labels(tags) for tags in train_y])

In [0]:
X_test = np.array([sentence_embodding(s) for s in test_x])
y_test = np.array([get_labels(tags) for tags in test_y])

X_dev = np.array([sentence_embodding(s) for s in dev_x])
y_dev = np.array([get_labels(tags) for tags in dev_y])

In [0]:
def labels_as_strings(vector_of_indices): 
    return ['irrelevant' if ind == 0 else 'relevant' for ind in vector_of_indices]

In [23]:
# SMOTE
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [25]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
print(log_reg.score(X_test, y_test))
print(classification_report(labels_as_strings(y_test), labels_as_strings(log_reg.predict(X_test))))

tn, fp, fn, tp = confusion_matrix(y_test, log_reg.predict(X_test)).ravel()
print("true negative: ", tn)
print("false positive: ", fp)
print("false negative: ", fn)
print("true positive: ", tp)

print("\n\n")

print(log_reg.score(X_dev, y_dev))
print(classification_report(labels_as_strings(y_dev), labels_as_strings(log_reg.predict(X_dev))))

tn, fp, fn, tp = confusion_matrix(y_dev, log_reg.predict(X_dev)).ravel()
print("true negative: ", tn)
print("false positive: ", fp)
print("false negative: ", fn)
print("true positive: ", tp)

0.6672512056115739
              precision    recall  f1-score   support

  irrelevant       0.96      0.65      0.78      2025
    relevant       0.22      0.80      0.35       256

    accuracy                           0.67      2281
   macro avg       0.59      0.73      0.56      2281
weighted avg       0.88      0.67      0.73      2281

true negative:  1317
false positive:  708
false negative:  51
true positive:  205



0.763396537510305
              precision    recall  f1-score   support

  irrelevant       0.99      0.76      0.86      1134
    relevant       0.20      0.87      0.32        79

    accuracy                           0.76      1213
   macro avg       0.59      0.81      0.59      1213
weighted avg       0.94      0.76      0.82      1213

true negative:  857
false positive:  277
false negative:  10
true positive:  69


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
